# **TASK**

Develop a Retrieval-Augmented Generation (RAG) model for a Question Answering (QA) bot that can
process financial terms and insights from a Profit & Loss (P&L) table extracted from PDF documents.
The QA bot should retrieve relevant information related to income, expenses, profit margins, and
other key financial metrics from the provided P&L table and generate accurate and coherent
responses.
Task Requirements:

● Implement a RAG-based model to handle questions related to a P&L table extracted from
PDF documents.

● Use a vector database (such as Pinecone) to store and retrieve document embeddings of
financial terms and data points efficiently.

● Parse P&L data from PDF documents into a structured format, such as tables or key-value
pairs, before storing embeddings.

● Test the model with several financial queries and show how accurately it retrieves and
generates responses from the dataset.


 **Installing all the necessary Python libraries for implementing the RAG model and related components**

 pinecone[grpc] and langchain-pinecone:
For storing and querying vector embeddings in the Pinecone vector database.

langchain, langchain-openai, langchain-text-splitters, and langchain-community:
For building the RAG pipeline using LangChain, enabling integrations with OpenAI models and text-splitting utilities for document processing.

In [1]:
!pip install \
pinecone[grpc] \
langchain-pinecone \
langchain-openai \
langchain-text-splitters \
langchain \
fastapi \
uvicorn \
requests \
langchain_community \
pydantic \
streamlit \
python-multipart \
pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4

In [2]:
import os
import getpass
os.environ["PINECONE_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········
··········


In [3]:
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough



**Importing libraries and configuring Pinecone client:
 This cell initializes the Pinecone vector database by importing necessary libraries and configuring the client for serverless mode**

In [4]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec, PodSpec
import time
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
use_serverless = True


**Creating the index and preparing Pinecone for use:
This cell creates a new Pinecone index with specified dimensionality (1536 for OpenAI's text-embedding-ada-002) and sets the similarity metric (e.g., cosine). It also ensures the index is ready before use by continuously checking its status until fully initialized.**

In [5]:
# configure client
pc = Pinecone()
if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-east-1')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec()
# check for and delete index if already exists
index_name = "sarvam-chat"
items = pc.list_indexes().indexes
existing_indexes = [item['name'] for item in items]
if index_name in existing_indexes:
    pc.delete_index(index_name)
    print("deleted_old")
# create a new index
pc.create_index(
    index_name,
    dimension=1536,  # dimensionality of text-embedding-ada-002
    metric='cosine',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)
index.describe_index_stats()

embeddings = OpenAIEmbeddings()
text_field = "text"

vectorstore = PineconeVectorStore(
    index, embeddings, text_field )


deleted_old


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Approach to Data Extraction and Preprocessing:**

In [8]:
#with open('iesc111.pdf','r') as file:
location = '/content/drive/My Drive/Sample Financial Statement.pdf'
pdf_loader = PyPDFLoader(location)

pages = pdf_loader.load_and_split()


**In this step, the financial data from Profit & Loss (P&L) tables stored in a PDF document is loaded using the PyPDFLoader. The PDF file is located on Google Drive, and the content is extracted and split into separate pages. This preprocessing step ensures that the P&L data is prepared for further analysis and embedding.**

In [9]:
pages[0]

Document(metadata={'source': '/content/drive/My Drive/Sample Financial Statement.pdf', 'page': 0, 'page_label': '1'}, page_content='Index Page No.\nCondensed Consolidated Balance Sheet ……………………………………………………………………………………………………………………..1\nCondensed Consolidated Statement of Profit and Loss ……………………………………………………………………………………………………………………..2\nCondensed Consolidated Statement of Changes in Equity ……………………………………………………………………………………………………………………..3\nCondensed Consolidated Statement of Cash Flows ……………………………………………………………………………………………………………………..5\nOverview and Notes to the Interim Condensed Consolidated Financial Statements\n1. Overview\n1.1 Company overview ……………………………………………………………………………………………………………………..7\n1.2 Basis of preparation of financial statements ……………………………………………………………………………………………………………………..7\n1.3 Basis of consolidation ……………………………………………………………………………………………………………………..7\n1.4 Use of estimates and judgments ……………………………………………………………………………………………………………………..7\n1.5 Critical accounting estimates and judgment

**Once the P&L data is extracted, it is embedded using the OpenAIEmbeddings model. This model converts the financial terms, metrics, and other information from the document into high-dimensional vectors that capture the semantic meaning of the data. These embeddings are stored in Pinecone, a vector database, allowing efficient similarity searches and retrieval of relevant information based on user queries. Like here we are adding the documments to our vectorstore**

In [10]:
# async_req is required to make sure it is updated in real time
vectorstore.add_documents(documents= pages, async_req=False)

['50d7dc6a-e95f-443b-8eee-8323040c6589',
 '068037fc-eaa6-4844-bd5d-9df597863665',
 'b28373cd-e917-4dcd-ae43-70dbadc0882b',
 'b05dcdd4-bb38-45d9-981c-3bcea0022d17',
 '3fdf5380-c289-4f9f-9220-3ff44014a155',
 '4669e453-4a3e-4107-ae43-05063a2052c8',
 'c1d7cb2e-a489-4d12-9bc9-34f938d2a989',
 '789395a3-fea8-4a27-9e0e-d4385b14e899',
 'e073ddf2-fa21-43c0-b457-7b4718ad109e',
 '536831e4-4305-49c2-81f3-0abc3300fbb7',
 '18b5642c-55f0-4050-8efd-e80c0568c34c',
 'e5f569ee-17ed-4d51-8d3e-4505fab4ee70',
 '48d6eebe-3933-4838-afd8-4c0a6b01a4f6',
 '38292caf-9b92-46ed-a25a-d1fe4cff8af1',
 '10abd77f-348e-428f-9ce9-5347cce908a5',
 '028aa116-e25b-44d8-8ba9-87216ac28ca3',
 'ea143e38-90a7-4722-9074-9ef797a338c7',
 '997c73cc-25e5-4e46-9717-5e16a336347a',
 'ceeeb320-066a-4922-bc11-4a242170fc5f',
 'db25a165-44f4-43be-a508-efec449da329',
 'ac3feb41-862a-4b49-9d8f-274aaac502a0',
 '8ca7257c-bc76-48f9-b50c-56b329f8070f',
 'eaf0e7d5-2a52-4be9-a7b0-e54d6d4502ef',
 'af56f6d8-485c-4640-86eb-634583016133',
 'fefb4773-7eb8-

**Query Definition: The variable query specifies the search query (in this case, "What is the book about?").
Search Operation: The vectorstore.similarity_search(query) retrieves documents from the vector store that are most similar to the query based on their embeddings.
Output (similar_docs): It stores the retrieved documents in similar_docs, which can then be used as contextual input for further processing, such as generating responses.**

In [11]:
query = "What is the book about?"

similar_docs = vectorstore.similarity_search(query)

In [12]:
similar_docs

[]

**Before passing the retrieved documents to the generative model,we format them into a readable form. This step ensures that the context is structured properly so that the model can understand and use it effectively to generate a meaningful response. The documents are joined together into a coherent format to serve as the input context for the RAG model.**

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

**Retrieval-Augmented Generation (RAG) pipeline to answer the question, "What is the gross profit for Q3 2024?". The process involves a retriever that searches for the top 6 relevant documents from the vector store based on the query.**

**A prompt template defines the role of the model (a financial analyst) and instructs it to answer only based on the provided context, otherwise responding with an apology if insufficient information is available**

**Finally, the RAG chain combines these steps: it retrieves relevant documents, formats them, and uses the GPT-4 model to generate a response based on the prompt and the retrieved context.**




In [23]:
question = "What is the gross profit for Q3 2024? How do the net income and operating expenses compare for Q1 2024?"
prompt = """You are a Financial Analyst. You answer queries based on Profit/Loss context provide to you.
    Use relevant information related to income, expenses, profit margins, and other key financial metrics from the provided context and generate accurate and coherent responses.
    Only provide answer based on the context information, else reply "I apologise. I do not have enough information to answer your query."
    """
retriever = vectorstore.as_retriever(search_kwargs={'k': 6})
prompt_rag = (
    PromptTemplate.from_template(prompt+"Answers given question: {question}. \nContext: {context}. In generated response provide reference to metadata from which context and table name along with column or row number or column and row name used in generating response")
)
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_rag
    | llm
)


res = rag_chain.invoke(question)

**Lastly we are printing our results**

In [24]:
print(res.content)

1. **What is the gross profit for Q3 2024?**

   The gross profit for Q3 2024 is ₹11,175 crore. This information is derived from the "FUNCTION WISE CLASSIFICATION OF CONDENSED CONSOLIDATED STATEMENT OF PROFIT AND LOSS" table, specifically from the row labeled "Gross profit" under the column for the year 2024.

2. **How do the net income and operating expenses compare for Q1 2024?**

   For Q1 2024, the net income is ₹7,975 crore, while the total operating expenses are ₹3,554 crore. This indicates that the net income is significantly higher than the operating expenses. The net income is found in the "FUNCTION WISE CLASSIFICATION OF CONDENSED CONSOLIDATED STATEMENT OF PROFIT AND LOSS" table, under the row "Profit for the period" for the year 2024, and the operating expenses are found in the same table under the row "Total operating expenses" for the year 2024.


In [16]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)

{'query': 'What is the book about?',
 'result': 'The document appears to be a set of financial statements and notes for Infosys Limited and its subsidiaries, prepared under Indian Accounting Standards (Ind AS) for the three months and year ended March 31, 2024. It includes various financial statements such as the balance sheet, statement of profit and loss, statement of changes in equity, and statement of cash flows, along with detailed notes on different financial aspects like business combinations, property, plant and equipment, financial instruments, and more.'}

# **DELIVERABLES**

# **Model Architecture:**

**The architecture is based on a Retrieval-Augmented Generation (RAG) model. It consists of the following key components:**

**Vector Embeddings: Financial terms and data points from the Profit & Loss (P&L) table are embedded into vector representations using the OpenAI embedding model. These embeddings capture the semantic meaning of the terms and metrics, allowing the system to efficiently search for related content.**

**Vector Database (Pinecone): The embeddings are stored in a Pinecone vectorstore. This allows efficient similarity searches across large datasets of financial terms, enabling quick retrieval of relevant data when a query is made.**

**Retriever: The retriever searches Pinecone's vectorstore to find relevant documents based on the query. It retrieves the most relevant data points, which are then formatted to provide context for the model.**

**Generative Model: The GPT-4 model is used for generating responses. The model receives a prompt along with the context retrieved from Pinecone, and it generates a coherent response based on the given data.**




#**How the Generative Responses are Created:**

**Query Processing:
When a user asks a question (e.g., "What is the gross profit for Q3 2024?"), the query is passed to the retriever, which uses Pinecone's similarity search to find the most relevant documents from the P&L data.**

**Context Generation:
The retrieved documents are then formatted into a cohesive context, which is combined with a prompt instructing the model to act as a financial analyst.**

**The formatted context and the user query are sent to the GPT-4 model. The model processes this information and generates a response based on the context provided. The model is trained to answer financial queries accurately using the data points retrieved from the P&L table.**

#**Challenges Encountered and Solutions Implemented:**

**Challenge: Extracting structured data from PDFs**

**Solution: The initial challenge was extracting data specially tabular data, in a structured manner from PDFs, as PDFs don't have a standardized format. Using tools like PyPDFLoader helped extract and split the text efficiently, although the formatting of the data still needed preprocessing for use in vector databases.**

**Challenge: Efficiently handling large datasets**

**Solution: Storing embeddings in Pinecone's vectorstore allowed efficient search and retrieval, ensuring that the system can handle large amounts of financial data without compromising on performance.**

**Challenge: Ensuring relevance in search results**

**Solution: By tweaking the parameters of the similarity search and adjusting the number of documents retrieved (k=6), the model was fine-tuned to retrieve the most relevant data for each query.**

**Challenge: Generating accurate and coherent financial responses**

**Solution: The model's prompt was carefully crafted to ensure that it could generate responses grounded in the provided financial context. Explicit instructions in the prompt helped ensure that the responses remained relevant and accurate, based on the data available.**
